# Test Model Recurrant Neural Network

In [1]:
# Clone the GitHub repository and Install Library
!git clone https://github.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch1.git
!pip install fastapi uvicorn pyngrok torch

Cloning into 'Hands-On-NLP-Super-Class-Batch1'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 136 (delta 45), reused 109 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (136/136), 3.67 MiB | 12.74 MiB/s, done.
Resolving deltas: 100% (45/45), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.5 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
from fastapi import FastAPI
from pydantic import BaseModel
import torch
import torch.nn as nn
import string
import unicodedata
import os
from pyngrok import ngrok
import uvicorn
from threading import Thread

In [3]:
# Define FastAPI app
app = FastAPI()

# Set up model configurations
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Define the path to the dataset in the cloned repo
data_path = 'Hands-On-NLP-Super-Class-Batch1/RNN/data/names'
all_categories = [os.path.splitext(filename)[0] for filename in os.listdir(data_path) if filename.endswith('.txt')]
n_categories = len(all_categories)

In [4]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        hidden = torch.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# Initialize and load the model
rnn = RNN(n_letters, 128, n_categories)
rnn.load_state_dict(torch.load("Hands-On-NLP-Super-Class-Batch1/RNN/models/rnn.pt"))
rnn.eval()

# Utility functions
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i]

def evaluate(line_tensor):
    hidden = rnn.initHidden()
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
    return output

# Define request and response models for FastAPI
class PredictionRequest(BaseModel):
    name: str

class PredictionResponse(BaseModel):
    category: str
    confidence: float

<ipython-input-4-5261a311ddfb>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rnn.load_state_dict(torch.load("Hands-On-NLP-Super-Class-Batch1/RNN/models/rnn.pt"))


In [5]:
# Prediction endpoint
@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    input_name = unicodeToAscii(request.name)
    input_tensor = lineToTensor(input_name)
    with torch.no_grad():
        output = evaluate(input_tensor)
        category = categoryFromOutput(output)
        confidence = torch.exp(output.max()).item()
    return PredictionResponse(category=category, confidence=confidence)

# Root endpoint
@app.get("/")
async def root():
    return {"message": "RNN Model is successfully running"}

In [8]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
# Run the server in a separate thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI server
thread = Thread(target=run)
thread.start()

Public URL: NgrokTunnel: "https://d2c3-34-74-81-47.ngrok-free.app" -> "http://localhost:8000"


In [11]:
import requests

url = "https://d2c3-34-74-81-47.ngrok-free.app/predict"
data = {"name": "Ikhwan"}
response = requests.post(url, json=data)
print(response.json())

INFO:     34.74.81.47:0 - "POST /predict HTTP/1.1" 200 OK
{'category': 'Dutch', 'confidence': 0.44768592715263367}
